# Data from Transfermarkt

In [ ]:
#https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
import pandas as pd
import requests # http requests
import re # regular expressions
from bs4 import BeautifulSoup # xml parsing



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_data_from_div(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the div with class 'responsive-table'
        div = soup.find('div', {'class': 'responsive-table'})
        
        if div:
            # Extract headers
            headers = div.find_all('th')
            header_texts = [header.get_text(strip=True) for header in headers]
            header_texts.append('Club')  # Adding 'Club' as a header
            
            # Extract contents
            rows = div.find_all('tr')
            for row in rows[2:]:  # Skipping the first row as it's the header row
                player_data = {}
                
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                
                # Checking if the row is complete (same length as headers)
                if len(cell_texts) == len(header_texts) - 1:
                    for index, header in enumerate(header_texts[:-1]):
                        player_data[header] = cell_texts[index]
                    
                    # Extracting the club name from the 'title' attribute
                    club_tag = cells[3].find('a')
                    if club_tag and 'title' in club_tag.attrs:
                        club_name = club_tag['title']
                        player_data['Club'] = club_name
                    
                    season_year = cell_texts[0][-2:]
                    if int(season_year) >= 8:
                        data.append(player_data)
                
        else:
            print('Div not found.')
    else:
        print('Failed to fetch data.')
    
    return data

messi = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
ronaldo = "https://www.transfermarkt.com/cristiano-ronaldo/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
result_messi = get_data_from_div(messi)

result_ronaldo = get_data_from_div(ronaldo)

# Convert to DataFrame
df_messi = pd.DataFrame(result_messi)
df_ronaldo = pd.DataFrame(result_ronaldo)

#print(df_messi)
#print(df_ronaldo)

In [ ]:
def get_tourna_stats(url):
    '''Gets tournament stats given player url'''
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # find all divs where class=responsive-table to get national, domestic, and international tables
        divs = soup.find_all('div', {'class': 'responsive-table'})

        print(len(divs))
        for idx, div in enumerate(divs):
            # defining headers in order
            header_texts = ['Season', 'Competition', 'Club', 'Appearances', 'Goals', 'Assists', 'Yellow Cards', 'Second yellow cards', 'Red cards', 'Minutes played']

            # extract contents
            rows = div.find_all('tr')
            for row in rows[2:]:#[1:]:  # Skipping the first row as it's the header row
                player_data = {}
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]
                del cell_texts[1] # remove empty cell due to league image

                # Extracting the club name from the 'title' attribute
                club_tag = cells[3].find('a')
                if club_tag and 'title' in club_tag.attrs:
                    club_name = club_tag['title']
                    cell_texts[2] = club_name

                # add is_national etc.
                header_texts.extend(['is_national', 'is_domestic', 'is_international'])
                if idx == 0:
                    cell_texts.extend([1, 0, 0])
                elif idx == 1:
                    cell_texts.extend([0, 1, 0])
                else:
                    cell_texts.extend([0, 0, 1])
                
                # add data
                for key, val in zip(header_texts, cell_texts):
                    player_data[key] = val

                season_year = cell_texts[0][-2:]
                if int(season_year) >= 8:
                    data.append(player_data)
    
    else:
        print('Failed to fetch data.')
    
    return data

In [ ]:
player_id = 8198 # ronaldo id
# player_id = 139208
url = f'https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{player_id}'
df = pd.DataFrame(get_tourna_stats(url))
df

In [ ]:
def get_trophy_stats(url):
    '''Gets award stats given player url'''
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    data = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # find all divs where class=responsive-table to get national, domestic, and international tables
        divs = soup.find_all('div', {'class': 'large-6 columns'})

        for idx, div in enumerate(divs):
            # defining headers in order
            header_texts = ['Season', 'Award']

            # extracting award header
            header_parts = div.find('h2').get_text(strip=True).split(' ')
            header_relevant_parts = header_parts[1:]
            header = ''
            for i in header_relevant_parts:
                header += i + ' '
            header = header.strip(' ')
            
            # extracting seasons/years awarded to player
            rows = div.find_all('tr')
            for row in rows:
                player_data = {}

                # rows can be just season, or season, club image, club
                cells = row.find_all('td')
                cell_texts = [cell.get_text(strip=True) for cell in cells]

                # if season is whole year use only last two digits
                season = cell_texts[0]
                if len(season) == 4:
                    season = season[-2:]
                cell_texts[0] = season

                # grab club/country if present and add data
                if len(cell_texts) > 1:
                    club_country = cell_texts[2]
                    player_data['Season'] = season
                    player_data['Club/Country'] = club_country
                    player_data['Award'] = header
                else:
                    player_data['Season'] = season
                    player_data['Award'] = header

                # keep only relevant years
                if int(season[-2:]) >= 8:
                    data.append(player_data)
                    
                    

            # for row in rows[2:]:#[1:]:  # Skipping the first row as it's the header row
            #     player_data = {}
            #     cells = row.find_all('td')
            #     cell_texts = [cell.get_text(strip=True) for cell in cells]
            #     del cell_texts[1] # remove empty cell due to league image

            #     # Extracting the club name from the 'title' attribute
            #     club_tag = cells[3].find('a')
                # if club_tag and 'title' in club_tag.attrs:
                #     club_name = club_tag['title']
                #     cell_texts[2] = club_name

                # # add is_national etc.
                # header_texts.extend(['is_national', 'is_domestic', 'is_international'])
                # if idx == 0:
                #     cell_texts.extend([1, 0, 0])
                # elif idx == 1:
                #     cell_texts.extend([0, 1, 0])
                # else:
                #     cell_texts.extend([0, 0, 1])
                
                # # add data
                # for key, val in zip(header_texts, cell_texts):
                #     player_data[key] = val

                # season_year = cell_texts[0][-2:]
                # if int(season_year) >= 8:
                #     data.append(player_data)
            #break
    
    else:
        print('Failed to fetch data.')
    
    return data

In [ ]:
player_id = 8198 # ronaldo id
# player_id = 139208
url = f'https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/{player_id}'
print(url)
df = pd.DataFrame(get_trophy_stats(url))
df

In [ ]:
df_ronaldo

In [ ]:
balondor_df = pd.read_csv("data/BallonDOr_combined.csv")
balondor_df

In [ ]:
#ballon_dor_df = pd.read_csv(r"BallonDOr.csv")
#print(ballon_dor_df["Year"]==2013)
excel_file = pd.read_csv("data/BallonDOr_combined.csv")
excel_file.loc[excel_file["Year"]>=2008]

In [ ]:
#print(ballon_dor_df["Year"])
ballon_2008_2018 = excel_file.loc[excel_file["Year"]>=2008]

p_08_18 = ballon_2008_2018["Player"].unique()
print(p_08_18)

In [ ]:
print(p_08_18[8])

In [ ]:
#!pip install --upgrade Selenium
#!pip install --upgrade ChromeDriver

In [ ]:
print(p_08_18[8].lower())

In [ ]:
import unicodedata

lower_list = []

for i in p_08_18:
    #j = i.split()
    #q = "-".join(j)
    j = i.lower()
    
    slug =  unicodedata.normalize('NFD', j).encode('ascii', 'ignore').decode('ascii')
    #print(slug)
    lower_list.append(slug)
    
    
print(lower_list)

In [ ]:
print(lower_list[8])

In [ ]:
#Player name, player_id
#Manually scraped becuase it takes more than 180h+ for jupyter notebook/code and risk of memory error if interent fails 
my_dict = {"cristiano ronaldo": 8198, "lionel messi": 28003, "fernando torres": 7767, "iker casillas": 3979,"xavi": 7607, 
           "andrei arshavin":15378, "david villa": 7980, "kaka": 3366, "zlatan ibrahimovic": 3455, "steven gerrard":3109,
          "marcos senna": 7840, "emmanuel adebayor":8883, "wayne rooney": 3332, "sergio aguero": 26399,
           "frank lampard": 3169,"franck ribery":22068, "samuel eto'o": 4257, "gianluigi buffon": 5023, 
           "cesc fabregas":8806, "michael ballack": 63, "sergio ramos": 25557, "nemanja vidic": 19726, 
           "didier drogba": 3924, "edwin van der sar": 3516, "ruud van nistelrooy": 3407,"andres iniesta": 7600, 
           "edin dzeko":28396, "ryan giggs": 3406, "thierry henry": 3207, "luis fabiano clemente": 19802,
           "diego forlan": 3408, "yoann gourcuff":18912, "julio cesar soares espindola": 22412, "maicon": 18301,
           "diego ribas da cunha": 4248, "john terry": 3160, "yaya toure": 13091,"wesley sneijder": 4673, 
           "xabi alonso":7476, "carles puyol": 7594, "mesut ozil": 35664, "arjen robben":4360, "thomas muller": 58358,
          "bastian schweinsteiger": 2514, "asamoah gyan": 26802,"julio cesar": 22412, "miroslav klose": 10,
           "philipp lahm": 2219, "dani alves": 15951,"luis suarez diaz":44352,"neymar": 68290, "eric abidal": 5283, 
          "karim benzema": 18922, "nani": 33706, "gerard pique": 18944, "radamel falcao": 39152, "andrea pirlo": 5817,
          "robin van persie": 4380,"manuel neuer": 17259, "sergio busquets": 65230, "mario balotelli":45416, 
          "gareth bale": 39381, "robert lewandowski": 38253, "edinson cavani": 48280, "thiago silva":29241, 
          "eden hazard": 50202, "james rodriguez": 88103, "toni kroos": 31909, "angel di maria": 45320, 
           "diego costa": 44779, "mario gotze": 74842, "javier mascherano": 19981,"thibaut courtois": 108390, 
           "paul pogba": 122153, "alexis sanchez":40433, "arturo vidal": 37666, "kevin de bruyne": 88755,
           "ivan rakitic": 32467, "antoine griezmann": 125781, "riyad mahrez": 171424, "jamie vardy":197838,
           "pepe": 14132, "pierre-emerick aubameyang": 58864, "rui patricio":45026, "luka modric": 27992, 
           "dimitri payet":37647, "paulo dybala":206050, "diego godin": 54928, "gonzalo higuain":39153,"koke": 74229,
           "hugo lloris": 17965, "kylian mbappe": 342229, "n'golo kante":225083, "harry kane":132098,"isco":85288,
           "marcelo": 44501, "david de gea": 59377, "leonardo bonucci": 39983, "sadio mane": 200512, "jan oblak": 121483,
           "mats hummels": 39728, "philippe coutinho": 80444, "dries mertens": 56416, "mohamed salah":148455, 
           "raphael varane": 164770, "roberto firmino":131789, "alisson": 105470, "mario mandzukic":34572, 
           "virgil van dijk": 139208, "bernardo silva": 241641,"frenkie de jong":326330, "raheem sterling":134425, 
           "matthijs de ligt": 326031, "trent alexander-arnold": 314353, "dusan tadic": 36139, "son heung-min":91845,
           "kalidou koulibaly": 93128, "marc-andre ter stegen": 74857, "georginio wijnaldum": 49499, "joao felix":462250,
           "marquinhos": 181767, "donny van de beek": 288255, "jorginho": 102017, "gianluigi donnarumma":315858,
           "erling haaland": 418560, "romelu lukaku":96341, "giorgio chiellini": 29260, "luis suarez":44352, 
           "simon kjr":48859, "simon kjaer": 48859, "mason mount": 346483, "bruno fernandes": 240306, 
           "lautaro martinez":406625, "pedri": 683840, "phil foden": 406635, "gerard moreno": 177467, 
           "nicolo barella": 255942, "ruben dias": 258004, "cesar azpilicueta": 57500, "vinicius junior": 371998,
           "sebastien haller":181375, "fabinho": 225693, "rafael leao": 357164, "casemiro": 16306,
           "luis diaz": 480692, "dusan vlahovic": 357498, "joao cancelo": 182712, "joshua kimmich": 161056,
           "mike maignan": 182906, "christopher nkunku": 344381, "darwin nunez": 546543, "antonio rudiger": 86202,
           "rodri": 357565, "julian alvarez": 576024, "victor osimhen": 401923, "yassine bounou":207834, "buno": 207834,
           "ilkay gundogan":53622, "emiliano martinez":111873, "khvicha kvaratskhelia": 502670, "jude bellingham":581678,
           "kim min-jae":503482, "kim min jae":503482, "andre onana": 234509, "bukayo saka":433177, 
           "josko gvardiol":475959, "josko gvardiol":475959, "jamal musiala":580195, "randal kolo muani":487969,
           "martin degaard": 316264, "martin-odegaard":316264}

##Took 1h and 30 min

In [ ]:
print(len(my_dict.keys()))

In [ ]:
len(ballon_2008_2018['Player'].unique())

In [ ]:
len(balondor_df['Player'].unique())

In [ ]:
### All the following strings are formatted with player_id in the actual code 
# Details_stats = "https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/8198/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    #Season, Competition Club, 15min intervals of scored goals, Ekstra time, Total amount of goals 
    
    
# Trophies = https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/34572
    # Kind of trophies, which year/season the specific kind of trophy was won in.


# National team = https://www.transfermarkt.com/mario-mandzukic/nationalmannschaft/spieler/34572/verein_id/3556/hauptwettbewerb//wettbewerb_id//start/2007-11-17/ende/2018-07-15/nurEinsatz/0/plus/1
    #Kind of match in which tournament, Club, Opponant, Date, Result, Goals, Assists, Own goals, 
    # Minute for what kind of card was given to the player, minutes played of the match

# Stats per tournament = https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/34572
    #Competition, Club name, appearances for the club, Goals, asissists, cards, Minutes Played
    #The aboed are mentioned for each type of legue (National legue, Domestic cups and International cups)

# Stats per club = https://www.transfermarkt.com/mario-mandzukic/leistungsdatenverein/spieler/34572
    #Club name, appearances for the club, Goals, asissists, cards, Minutes Played, 
    #The above are mentioned for each league the player played in 


players = []
player_dfs = []
tourna_dfs = []
trophy_dfs = []
for year in range(2008,2024): 
    #print(year)
    m = ballon_2008_2018.loc[ballon_2008_2018["Year"]== year]
    
    for i in m["Player"].values: 
        slug =  unicodedata.normalize('NFD', i).encode('ascii', 'ignore').decode('ascii')
        lowered = slug.lower()
        try:
            if lowered not in players:
                players.append(lowered)
                player_id = my_dict[lowered]
                print(lowered, player_id)
                new_url = f"https://www.transfermarkt.com/lionel-messi/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
        
                #print(lowered, my_dict[lowered])
                
                result_player = get_data_from_div(new_url)
                df_player = pd.DataFrame(result_player)
                df_player['Competition'] = df_player['reihenfolge ASC, saison_id DESC, verein_id ASC']
                df_player.drop('reihenfolge ASC, saison_id DESC, verein_id ASC', axis=1, inplace=True)
                df_player['Player'] = [i] * df_player.shape[0]
                player_dfs.append(df_player)

                tourna_url = f'https://www.transfermarkt.com/mario-mandzukic/detaillierteleistungsdaten/spieler/{player_id}'
                df_tourna = pd.DataFrame(get_tourna_stats(tourna_url))
                df_tourna['Player'] = [i] * df_tourna.shape[0]
                tourna_dfs.append(df_tourna)

                trophy_url = f'https://www.transfermarkt.com/mario-mandzukic/erfolge/spieler/{player_id}'
                df_trophy = pd.DataFrame(get_trophy_stats(trophy_url))
                df_trophy['Player'] = [i] * df_trophy.shape[0]
                trophy_dfs.append(df_trophy)


                #print(df_player)
               # break
                
        except: 
            pass
       # break
        #result_zlatan = get_data_from_div(zlatan)

        # Convert to DataFrame
        #df_zlatan = pd.DataFrame(result_zlatan)
        

In [ ]:
# season date should end with the last two year digits (8, 9, 10, ..., 23)

In [ ]:
print(tourna_dfs.shape[0])
print(player_dfs)

In [ ]:
for i in tourna_dfs:
    print(len(i.columns))

In [ ]:
# joining all tourna dfs
df = pd.concat(tourna_dfs)

In [ ]:
df.to_csv('data/tournament_stats.csv', index=False)

In [ ]:
for i in trophy_dfs:
    print(i.columns)

In [ ]:
# joining all trophy dfs
df = pd.concat(trophy_dfs)
df.to_csv('data/awards.csv', index=False)

In [ ]:
for i in player_dfs:
    print(i.columns)

In [ ]:
# joining all player dfs
df = pd.concat(player_dfs)

# replace NaN values in Extra time column
df['Extra time'] = df['Extra time'].fillna('-')

In [ ]:
df.to_csv('data/goals.csv', index=False)

In [ ]:
len(df_read['Competition'].unique())

In [ ]:
len(df_read['Club'].unique())

In [ ]:
print(df_read['Competition'].unique())

In [ ]:
df_read = pd.read_csv('data/details_stats.csv')
df_read

In [ ]:
df_player
# df_player = df_player.iloc[1:]
df_player['Competition'] = df_player['reihenfolge ASC, saison_id DESC, verein_id ASC']
df_player.drop('reihenfolge ASC, saison_id DESC, verein_id ASC', axis=1, inplace=True)
df_player

In [ ]:
#Zlatan Ibrahimović
base_url = "https://www.transfermarkt.com/zlatan-ibrahimovic/profil/spieler/1"
p = base_url.split("/")[-4]
print(" ".join(p.split("-")))

In [ ]:
99800/60/24/10

In [ ]:
"""
from selenium import webdriver

base_url = "https://www.transfermarkt.com/silvio-adzic/profil/spieler/"
# Initialize the Chrome WebDriver
browser = webdriver.Chrome()


important_player_ids = {}

for player_id in range(0, 99998):
    url = base_url + str(player_id)
    try:
        browser.get(url)
        
        # Wait for a certain time (you can use other waits if needed)
        browser.implicitly_wait(0.1)

        # Get the current URL after the delay
        final_url = browser.current_url

        #print(final_url)
        p = final_url.split("/")[-4]
        string = " ".join(p.split("-"))
        #print(string, lower_list[8])
        
        if string in lower_list:
            print(string)
            # If found, perform actions or extract information here...
            
            # Stop the loop if Zlatan Ibrahimovic is found
            important_player_ids[p] = player_id
            
        
    except Exception as e:
        print("Exception occurred:", str(e))
    finally:
        # Close the browser after all iterations are done
        if player_id == 99997:  # Check if it's the last iteration
            browser.quit()
        # To avoid quitting the browser prematurely in other iterations

print(important_player_ids)
print(lower_list)
"""

In [ ]:
zlatan = "https://www.transfermarkt.com/Zlatan-Ibrahimović/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"

result_zlatan = get_data_from_div(zlatan)

# Convert to DataFrame
df_zlatan = pd.DataFrame(result_zlatan)


neymar = "https://www.transfermarkt.com/Neymar/torenachminute/spieler/28003/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"

result_neymar = get_data_from_div(neymar)

# Convert to DataFrame
df_neymar = pd.DataFrame(result_neymar)




In [ ]:
print(df_neymar)